# FIT5196 Task 2 in Assessment 1
#### Student Name: Syed Ali Alim Rizvi
#### Student ID: 28984773

Date: 08/04/2018

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas: for dataframe
* pdftables: to instially convert and parse and the pdf to text in table form. 
* copy: inorder to create a deep copy of the tables extracted.
* re: for regexes. 

<font size=3, color="red"> Note: this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. It is important to make sure the logic of how you finish the assessment is clearly shown in this notebook! </font>

### Parsing PDF using pdftables

#!pip install pdftables.six
#!pip install numpy --upgrade
#!pip install pandas
#!pip install copy
#!pip install re

## 1.  Import libraries 

In [1]:
#required imports
import re
import pandas as pd
from copy import deepcopy
from pdftables import get_tables


Parse the pdf file using pdftables. This creates an object by pdftables that can then be used to extract tables. The tables here represent pdf pages as each page is parsed as a table.

## 2. Parse PDF File

In this section, you can write your python scripts to parse the correspondiing file.
You should 
* write proper notes for all code block in this notebook using the Markdown cells
* provide proper comment in your scripts
* run all cells to make sure scripts are runable. If the scripts cannot be run by the assessors, they will not be assessed and zero mark will be given to the task.

In [2]:
#parse pdf file
pdfFile = 'health.pdf'
pdfobj = open(pdfFile, 'rb')
tables = get_tables(pdfobj)

In [3]:
for table in tables:
    for row in table:
        print(row)
    print('\n==================================end of page=========================================\n')

['TABLE 3. H', 'EA', 'LT', 'H', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', 'I', 'mmuni', 'zation c', 'overage', '(%)', '', '', '', 'Pneumonia', 'Diarrhoea', '', 'Malaria', '']
['', 'Udris', 'se of bnking ervices', 'asic water  (%)', 'Use of bsanitatiservices', 'asic on  (%)', '', '', '', '', '', '', '', '', '', '', 'Protection at birth (PAB) against', 'Care seekingfor children with  symptoms of', 'Treatment with oral rehydra-tion salts', 'Care seeking for children with fever', 'Children sleep-ing under', 'House-holds with at least one']
['Countries', '', '2015', '', '2015', '', 'BCG', 'DTP1β', 'DTP3β', 'polio3', 'MCV1', 'MCV2^', 'HepB3', 'Hib3', 'rota', 'PCV3', 'tetanusλ', '(%)pneumonia', '(ORS) (%)', '(%)', 'ITNs (%)', 'ITN (%)']
['and areas', 'total', 'urban', 'rural', 'totalurban', 'rural', '', '', '', '', '', '2016', '', '', '', '', '', '', '2', '011–2016*', '', '']
['Afghanistan', '63', '89', '53', '3956', '33', '74

## Key observations that would help in cleaning the data:

##### Filtering rows to get only required data

* The data on each page starts from index 5. This implies when reading the table we have to skip the unwanted rows
```python
    for row in table[5:]
```
* The data required ends in two ways. 

    * For pages 1-3 the last two rows consist of empty strings `''`; hence we can stop extracting rows when we hit this empty row.
    * For the last page (page 4) the data ends at the country name `Zimbabwe`; hence we can strong extracting when we reach this row. 
    

    for page 1-3:
```
    ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
    ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
``` 
    for page 4
```
    ['Zimbabwe679454', '395431959490', '90', '95', '63', '90', '90', '919080', '51', '4150', '9', '48']

```

Hence the code would some what look like the following:

```python
    new_tables = deepcopy(tables) #so that we dont damage the original data
    
    for page, table in enumerate(new_tables):
        for row in table[5:]:
            if page != 3:
                print(row)
                if row[0] == '':
                    break
            else:
                print(row)
                if 'Zimbab' in row[0]:
                    break
    

```


##### Column cleaning:

We can see that during the parsing process pdf tables has read a few columns into one. The good thing for us is that there is a pattern to this mess for each page (table). The patter is as follows:

* For pages 1 and 3 columns five and six have been merged together. 

    *Page 1:*
    
        ['Afghanistan', '63', '89', '53', '3956', ...
        ['Albania', '91', '93', '90', '9898', ...
    
    *Page 3:*
    
        ['Netherlands', '100', '100', '100', '9898', ...    
        ['New Zealand', '100', '100', '100', '100100', ...

* For page 2 columns six and seven have been merged together. 
    
    *Page 2:*
    
        ['France', '100', '100', '100', '99', '9999', ...
        ['Gabon', '88', '92', '59', '41', '4232', ...
        

since the values vary from 0-100 and can be a '-'; 

        '100100'
        '66–'
        '21'

we create a regex as follows:
```python
   nbr_grp = '(–|0|100|[1-9][0-9]?)' #to catch one number or a dash

   reg0 = reg.compile(r'^{}{}$'.format(nbr_group, nbr_group))
```

hence we can split the columns with the following code:

```python
    #create a new deep copy to keep original table unedited
    new_tables = deepcopy(tables)

    #loop over each page in tables
    for page, table in enumerate(new_tables):
        #loop over rows
        for row in table[5:]:
            #do this for pages 1,2, and 3 but not for 4
            if page != 3:
                #break loop of page if data rows end
                if row[0] == '':
                    break
                
                *** cleaning code starts ***

                #check page as to know which column to split
                ind = 4 if page in [0,2] else 5

                #the col to fix
                fix = row[ind]

                #del the column
                del row[ind]

                #extract correct values from dirty col
                reg0 = reg.match(fix)

                #insert correct values in correct cols
                if reg is not None:
                    row.insert(ind, reg0.group(2))
                    row.insert(ind+1, reg0.group(3))
                else:
                    break #to know if regex didnt work
                    
                *** cleaning code ends ***

            #do this for page 4
            else:
                #if the current row is the last row of data break the loop
                if 'Zimbab' in row[0]:
                    break
            
            #print row to see output
            print(row)
            
               
```

* The last page requires a bit more work. the following are the issues
    * Index **0** has columns 1,2,3, and 4 are merged together. This means the name of the country and three numeric values are merged. 
```   
    ['Uganda397332', ...
    ['Ukraine9897100', ...
    ['Venezuela (Bolivarian Republic of)979986', ...
```

    For this we use a new regex:
```python
    name_grp = '([A-Za-z() ]*)' #extracts names (if any: the '*' allows zero occurences)
```
    *  Index **1** has six cols numerics merged, Index **7** has three cols numerics merged and index **9** has two cols numerics merged.
    
Also since index **9** has an 'x' attached to the value we will need to update the regex.
        
        '38x–'
        
Hence we create a general regex that can be used for all extractions and provide it with group for each access of values:
```python
    reg = re.compile(r'^{}{}x?{}{}?{}?{}?{}?$'.format(name_grp, nbr_grp, nbr_grp, nbr_grp, nbr_grp, nbr_grp, nbr_grp))
```

We then add it to our existing code to get the following:

```python
#create a new deep copy to keep original table unedited
new_tables = deepcopy(tables)

#loop over each page in tables
for page, table in enumerate(new_tables):
    #loop over rows
    for row in table[5:]:
        #do this for pages 1,2, and 3 but not for 4
        if page != 3:
            #break loop of page if data rows end
            if row[0] == '':
                break

            #check page as to know which column to split
            ind = 4 if page in [0,2] else 5
            #the col to fix
            fix = row[ind]
            #del the column
            del row[ind]
            #extract correct values from dirty col
            reg0 = reg.match(fix)
            #insert correct values in correct cols
            if reg is not None:
                row.insert(ind, reg0.group(2))
                row.insert(ind+1, reg0.group(3))
            else:
                break
        
        #do this for page 4
        else:
            
            *** New cleaning ***
                            
            #store dirty values
            fix1 = row[0]
            fix2 = row[1]
            fix8 = row[7]
            fix10 = row[9]

            #del dirty values from row since we have now stored them
            for ind in [-12, -11, -5, -3]:
                del row[ind]

            #extract correct values from dirty values
            reg1  = reg.match(fix1)
            reg2  = reg.match(fix2)
            reg8  = reg.match(fix8)
            reg10 = reg.match(fix10)

            #put correct vals from dirty col 1
            for grp, ind in enumerate(range(4),1):
                row.insert(ind, reg1.group(grp))

            #put correct vals from dirty col2
            for grp, ind in enumerate(list(range(4,10)), 2):
                row.insert(ind, reg2.group(grp))

            #put correct vals from dirty col 8
            for grp, ind in enumerate(range(15,18), 2):
                row.insert(ind, reg8.group(grp))

            #put correct vals from dirty col 10
            for grp, ind in enumerate(range(19,21), 2):
                row.insert(ind, reg10.group(grp))
                                                
            *** Cleaning ends here ***

            #if the current row is the last row of data break the loop
            if 'Zimbab' in row[0]:
                break
        
        #print the cleaned row
        print(row)
```

Our code is now almost done. Only 2 steps left. first we need to clean any 'x's next to our values or any '-'s. for that we use a simple for loop:

```python
        #remove any '-' and 'x' in row values
        for ind in range(1, len(row)):
            if 'x' in row[ind]:
                row[ind] = row[ind].replace('x', '')
            elif '–' in row[ind]:
                row[ind] = row[ind].replace('–', '')
```

##### Creating a dictionary to store clean data


Our final step is to store it into a dictionary that can be done by creating an empty dictionary in the beginning and adding the clean row at the end. 

```python

#create empty dictionary to store clean data in
clean_data = {}
.
.
.
#store the data into a dictionary
clean_data[row[0]] = row[1:]
```

In [4]:
#regexes to clean last table
name_grp = '([A-Za-z() ]*)' #regex group to extract name from a dirty column
nbr_grp = '(–|0|100|[1-9][0-9]?)' #regex to extract a number from a dirty merged column

#combination of above regexes for cleaning columns 1,2,8,10
reg = re.compile(r'^{}{}x?{}{}?{}?{}?{}?$'.format(name_grp, nbr_grp, nbr_grp, nbr_grp, nbr_grp, nbr_grp, nbr_grp))

In [5]:
#create a new deep copy to keep original table unedited
new_tables = deepcopy(tables)

#create empty dictionary to store clean data in
clean_data = {}

#loop over each page in tables
for page, table in enumerate(new_tables):

    #loop over rows
    for row in table[5:]:
        
        #do this for pages 1,2, and 3 but not for 4
        if page != 3:

            #break loop of page if data rows end
            if row[0] == '':
                break

            #check page as to know which column to split
            ind = 4 if page in [0,2] else 5

            #the col to fix
            fix = row[ind]

            #del the column
            del row[ind]

            #extract correct values from dirty col
            reg0 = reg.match(fix)

            #insert correct values in correct cols
            if reg is not None:
                row.insert(ind, reg0.group(2))
                row.insert(ind+1, reg0.group(3))
            else:
                break
        
        #do this for page 4
        else:
            
            #store dirty values
            fix1 = row[0]
            fix2 = row[1]
            fix8 = row[7]
            fix10 = row[9]

            #del dirty values from row since we have now stored them
            for ind in [-12, -11, -5, -3]:
                del row[ind]

            #extract correct values from dirty values
            reg1  = reg.match(fix1)
            reg2  = reg.match(fix2)
            reg8  = reg.match(fix8)
            reg10 = reg.match(fix10)

            #put correct vals from dirty col 1
            for grp, ind in enumerate(range(4),1):
                row.insert(ind, reg1.group(grp))

            #put correct vals from dirty col2
            for grp, ind in enumerate(list(range(4,10)), 2):
                row.insert(ind, reg2.group(grp))

            #put correct vals from dirty col 8
            for grp, ind in enumerate(range(15,18), 2):
                row.insert(ind, reg8.group(grp))

            #put correct vals from dirty col 10
            for grp, ind in enumerate(range(19,21), 2):
                row.insert(ind, reg10.group(grp))

            #if the current row is the last row of data break the loop
            if 'Zimbab' in row[0]:
                break
        
        #remove any '-' and 'x' in row values
        for ind in range(1, len(row)):
            if 'x' in row[ind]:
                row[ind] = row[ind].replace('x', '')
            elif '–' in row[ind]:
                row[ind] = row[ind].replace('–', '')
        
        #print the cleaned row
        print(row)
        
        #store the data into a dictionary
        clean_data[row[0]] = row[1:]

['Afghanistan', '63', '89', '53', '39', '56', '33', '74', '73', '65', '60', '62', '39', '65', '65', '0', '65', '65', '62', '46', '63', '5', '26']
['Albania', '91', '93', '90', '98', '98', '97', '99', '99', '98', '98', '96', '98', '98', '98', '0', '98', '92', '70', '54', '71', '', '']
['Algeria', '93', '95', '89', '87', '90', '82', '99', '96', '91', '91', '94', '96', '91', '91', '0', '61', '92', '66', '25', '', '', '']
['Andorra', '100', '100', '100', '100', '100', '100', '', '99', '98', '98', '97', '90', '94', '98', '0', '92', '', '', '', '', '', '']
['Angola', '41', '63', '23', '39', '62', '21', '58', '79', '64', '66', '49', '26', '64', '64', '53', '58', '78', '49', '43', '51', '22', '31']
['Anguilla', '98', '98', '', '97', '97', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['Antigua and Barbuda', '97', '', '', '88', '', '', '', '99', '99', '86', '98', '87', '99', '99', '0', '0', '', '', '', '', '', '']
['Argentina', '100', '100', '100', '95', '95', '94', '92', 

All thats left is to transfer that data into a data frame using the dictionary created. Transpose it to get the correct format and export it as a csv file. 

In [6]:
#create dataframe from dictionary and transpose it
df = pd.DataFrame(clean_data)
df = df.T
df.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Afghanistan,63,89,53,39,56,33,74,73,65,60,...,65,65,0,65,65,62,46,63,5,26
Albania,91,93,90,98,98,97,99,99,98,98,...,98,98,0,98,92,70,54,71,,
Algeria,93,95,89,87,90,82,99,96,91,91,...,91,91,0,61,92,66,25,,,
Andorra,100,100,100,100,100,100,,99,98,98,...,94,98,0,92,,,,,,
Angola,41,63,23,39,62,21,58,79,64,66,...,64,64,53,58,78,49,43,51,22,31


In [7]:
#label the index column
df.index.name = 'Country Name'

In [8]:
#create a csv file
df.to_csv('health.csv')

## 3. Summary
Give a short summary of your work done above, such as your findings.

Extracting using pdftables was convenient as rather than one text doc it provided row by row extracts and divided the pages into different tables.

Removing the dirty rows that were not needed were not much of an issue but the fact that pdf table had read in the file by merging a few columns together was a major challenge. Even after using regexes to extract the given data the issue was that since the values of the columns were ranging in the number of digits it consisted of, the resulted parsed and cleaned data still had a chance of error as the regex cannot automatically create a distinction between which value to keep as one digit or two. 

For example in a case where two columns are merged. A numerical value of 999 would create issues as we dont nescessarily know if the values were "9, 99" or "99, 9". Hence this increased the probable level of error in the parsed file. 